In [ ]:
!pip install tiktoken

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
# import importlib
import tiktoken
print(f"tiktoken version: {tiktoken.__version__}")

tiktoken version: 0.12.0


In [ ]:
with open('the-verdict.txt', 'r') as f:
    text = f.read()

print(len(text))

20479


In [ ]:
text

'I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no great surprise to me to hear that, in the height of his glory, he had dropped his painting, married a rich widow, and established himself in a villa on the Riviera. (Though I rather thought it would have been Rome or Florence.)\n\n"The height of his glory"--that was what the women called it. I can hear Mrs. Gideon Thwing--his last Chicago sitter--deploring his unaccountable abdication. "Of course it\'s going to send the value of my picture \'way up; but I don\'t think of that, Mr. Rickham--the loss to Arrt is all I think of." The word, on Mrs. Thwing\'s lips, multiplied its _rs_ as though they were reflected in an endless vista of mirrors. And it was not only the Mrs. Thwings who mourned. Had not the exquisite Hermia Croft, at the last Grafton Gallery show, stopped me before Gisburn\'s "Moon-dancers" to say, with tears in her eyes: "We shall not look upon its like again"?\n\nWell!--even 

In [ ]:
import re
tokenized_text = re.split(r'[,.:"()?\_!]|--|\s', text)
print(len(tokenized_text))

4502


In [ ]:
# tokenized_text

In [ ]:
tokenized_optimize = [token.strip() for token in tokenized_text if token.strip()]
print(len(tokenized_optimize))
# print(tokenized_optimize)

3712


## donot remove whitespace if we are trainning model for coding or that is sensetive to whitespace

# Vocabloury

In [ ]:
vocab_unique = set(tokenized_optimize)
vocab = sorted(vocab_unique)

In [ ]:
vocab[:10]

["'",
 "'Are",
 "'It's",
 "'coming'",
 "'done'",
 "'subject",
 "'technique'",
 "'way",
 ';',
 'A']

In [ ]:
vocab_size = len(vocab)
print(vocab_size)

1157


In [ ]:
vocab_dict = {token: index for index, token in enumerate(vocab)}#enumerate returns index and token
print(vocab_dict)

{"'": 0, "'Are": 1, "'It's": 2, "'coming'": 3, "'done'": 4, "'subject": 5, "'technique'": 6, "'way": 7, ';': 8, 'A': 9, 'Ah': 10, 'Among': 11, 'And': 12, 'Arrt': 13, 'As': 14, 'At': 15, 'Be': 16, 'Begin': 17, 'Burlington': 18, 'But': 19, 'By': 20, 'Carlo': 21, 'Carlo;': 22, 'Chicago': 23, 'Claude': 24, 'Come': 25, 'Croft': 26, 'Destroyed': 27, 'Devonshire': 28, "Don't": 29, 'Dubarry': 30, 'Emperors': 31, 'Florence': 32, 'For': 33, 'Gallery': 34, 'Gideon': 35, 'Gisburn': 36, "Gisburn's": 37, 'Gisburns': 38, 'Grafton': 39, 'Greek': 40, 'Grindle': 41, "Grindle's": 42, 'Grindles': 43, 'HAD': 44, 'Had': 45, 'Hang': 46, 'Has': 47, 'He': 48, 'Her': 49, 'Hermia': 50, "Hermia's": 51, 'His': 52, 'How': 53, 'I': 54, "I'd": 55, "I'll": 56, "I've": 57, 'If': 58, 'In': 59, 'It': 60, "It's": 61, 'Jack': 62, "Jack's": 63, 'Jove': 64, 'Just': 65, 'Lord': 66, 'Made': 67, 'Miss': 68, "Money's": 69, 'Monte': 70, 'Moon-dancers': 71, 'Mr': 72, 'Mrs': 73, 'My': 74, 'Never': 75, 'No': 76, 'Now': 77, 'Nutley':

In [ ]:
class tokenizer:
    def __init__(self, vocab_dict):
        self.token_index =  vocab_dict#{token:index}
        self.index_token = {index: token for token, index in vocab_dict.items()} # {}

    def encoder(self,text):
        preprocessed_text= re.split(r'[,.:"()?\_!]|--|\s', text)
        tokens = [token.strip() for token in preprocessed_text if token.strip()]
        ids = [self.token_index[token] for token in tokens]
        return ids
    
    def decoder(self,ids):
        text = " ".join([self.index_token[id] for id in ids])
        text = re.sub(r'\s+([.,:;!?])', r'\1', text)
        # print(text)
        return text

In [ ]:
tokenized_obj = tokenizer(vocab_dict)
# ids = tokenized_obj.encoder(text)
# print(ids)
# print(tokenized_obj.decoder(ids))


In [ ]:
text = """It's the last he painted, you know,"Mrs.Gisburn said with pardonable pride."""
ids = tokenized_obj.encoder(text)
print(ids)


[61, 1008, 622, 545, 766, 1151, 615, 73, 36, 872, 1133, 776, 815]


In [ ]:
print(tokenized_obj.decoder(ids))

It's the last he painted you know Mrs Gisburn said with pardonable pride


In [ ]:
test_text = "hello, what is your name?"
ids = tokenized_obj.encoder(test_text)
print(ids)
# print(tokenized_obj.decoder(ids))

KeyError: 'hello'

In [ ]:
# adding special token context
all_tokens = sorted(list(set(tokenized_optimize)))
all_tokens.extend(['<|endoftext|>', '<|unk|>'])
vocab_v2 = {token: index for index, token in enumerate(all_tokens)}
print(vocab_v2)

{"'": 0, "'Are": 1, "'It's": 2, "'coming'": 3, "'done'": 4, "'subject": 5, "'technique'": 6, "'way": 7, ';': 8, 'A': 9, 'Ah': 10, 'Among': 11, 'And': 12, 'Arrt': 13, 'As': 14, 'At': 15, 'Be': 16, 'Begin': 17, 'Burlington': 18, 'But': 19, 'By': 20, 'Carlo': 21, 'Carlo;': 22, 'Chicago': 23, 'Claude': 24, 'Come': 25, 'Croft': 26, 'Destroyed': 27, 'Devonshire': 28, "Don't": 29, 'Dubarry': 30, 'Emperors': 31, 'Florence': 32, 'For': 33, 'Gallery': 34, 'Gideon': 35, 'Gisburn': 36, "Gisburn's": 37, 'Gisburns': 38, 'Grafton': 39, 'Greek': 40, 'Grindle': 41, "Grindle's": 42, 'Grindles': 43, 'HAD': 44, 'Had': 45, 'Hang': 46, 'Has': 47, 'He': 48, 'Her': 49, 'Hermia': 50, "Hermia's": 51, 'His': 52, 'How': 53, 'I': 54, "I'd": 55, "I'll": 56, "I've": 57, 'If': 58, 'In': 59, 'It': 60, "It's": 61, 'Jack': 62, "Jack's": 63, 'Jove': 64, 'Just': 65, 'Lord': 66, 'Made': 67, 'Miss': 68, "Money's": 69, 'Monte': 70, 'Moon-dancers': 71, 'Mr': 72, 'Mrs': 73, 'My': 74, 'Never': 75, 'No': 76, 'Now': 77, 'Nutley':

In [ ]:
class tokenizer_v2:
    def __init__(self, vocab_dict):
        self.token_index =  vocab_dict#{token:index}
        self.index_token = {index: token for token, index in vocab_dict.items()} # {}

    def encoder(self,text):
        preprocessed_text= re.split(r'[,.:"()?\_!]|--|\s', text)
        tokens = [token.strip() for token in preprocessed_text if token.strip()]
        tokens = [
          token if token in self.token_index else '<|unk|>' for token in tokens
        ]
        ids = [self.token_index[token] for token in tokens]
        return ids

    def decoder(self,ids):   
        text = " ".join([self.index_token[id] for id in ids])
        text = re.sub(r'\s+([.,:;!?])', r'\1', text)
        return text
     

In [ ]:
test_1 = "hello, what is your name?"
test_2 = "my name is mahesh"

test_text = "<|endoftext|> ".join([test_1, test_2])
tokenized_obj_v2 = tokenizer_v2(vocab_v2);
ids = tokenized_obj_v2.encoder(test_text)
print(ids)
print(tokenized_obj_v2.decoder(ids))

[1158, 1112, 601, 1155, 1158, 1157, 717, 1158, 601, 1158]
<|unk|> what is your <|unk|> <|endoftext|> my <|unk|> is <|unk|>


# Byte Pair Encoding
<div class="bpe-card">
  <h1>Why do we need BPE?</h1>
  <p>
    In word-based tokenization, we often run into the <strong>out-of-vocabulary (OOV)</strong> problem.
    Words that look similar can be treated as completely different tokens, even though they share meaning.
    <br /><br />
    <strong>Byte Pair Encoding (BPE)</strong> solves this by breaking words into subwords, allowing the model
    to handle unseen words and capture shared structure between related terms.
  </p>
</div>

<style>
  .bpe-card {
    max-width: 700px;
    padding: 20px 24px;
    margin: 20px auto;
    background: #f9fafb;
    border-left: 5px solid #22c55e;
    border-radius: 8px;
    font-family: system-ui, -apple-system, BlinkMacSystemFont, sans-serif;
  }

  .bpe-card h1 {
    margin-bottom: 12px;
    color: #166534;
    font-size: 1.8rem;
  }

  .bpe-card p {
    color: #374151;
    line-height: 1.6;
    font-size: 1rem;
  }
</style>


# Types of Tokenization

## 1. Word-based Tokenization
- Each word is treated as a token.

### Example
"boys are playing" → ["boys", "are", "playing"]

y

### Problems
- Very large vocabulary size
- Out-Of-Vocabulary (OOV) problem
- Similar words are treated as unrelated  
  (e.g., `play`, `playing`, `played`)

---

## 2. Character-based Tokenization
- Each character is a token.

### Example
"boys" → ["b", "o", "y", "s"]

markd

### Advantages
- Very small vocabulary
- No OOV problem
- Low memory required for vocabulary

### Problems
- Word meaning is mostly lost
- Token sequence length is very large
- Slower training and inference

### Solution
➡️ **Subword-based Tokenization**

---

## 3. Subword-based Tokenization
- Frequent words are **not split**
- Rare words are split into **smaller meaningful subwords**

### Example
boys → boy + s
playing → play + ing

y

### Advantages
- Handles OOV words
- Preserves meaning via shared root words
- Smaller vocabulary than word-based tokenization
- Shorter sequences than character-based tokenization
- Better generalization

---

## Common Subword Tokenization Methods
- Byte Pair Encoding (BPE)
- WordPiece
- Unigram Language Model (SentencePiece)

---

## Byte Pair Encoding (BPE)
- Originally developed as a **data compression algorithm**
- Widely used in **Large Language Models (LLMs)** for tokenization

---

## BPE Algorithm (Compression Example)

### Input
aaabdaaabac

markd

### Step 1
- Most frequent pair: `aa`
- Replace with `Z`

ZabdZabac

markd

### Step 2
- Most frequent pair: `ab`
- Replace with `Y`

ZYdZYac

sh

### Step 3
- Remaining pairs are rare → stop

### Final Compressed Output
ZYdZYac

y

---

## How BPE Is Used in LLMs

> LLMs do **not** replace characters with random symbols.  
> Instead, they **merge frequent character pairs into subword tokens** and store them in a vocabulary.

---

## BPE for LLMs – Step-by-Step Example

### Dataset
```json
{"old": 7, "older": 3, "finest": 9}
Step 1: Initialize with Characters

o l d
o l d e r
f i n e s t
Each word keeps its frequency.

Step 2: Find Frequent Pairs
Examples:

o l

l d

e r

e s

s t

Assume the most frequent pair is:


o + l → ol
Step 3: Merge Repeatedly
Possible merges:


ol + d → old
e + r → er
es + t → est
Final Tokenization

old
older → old + er
finest → fin + est
Why BPE Works Well for LLMs
Reduces OOV problem

Learns common roots automatically

Efficient vocabulary size

Balances meaning and sequence length

Language independent

One-Line Summary
BPE is a subword tokenization algorithm that iteratively merges frequent symbol pairs, allowing LLMs to efficiently handle rare and unseen words while preserving semantic structure.


In [ ]:
# implimenting BPE in Python
tokenizer  = tiktoken.get_encoding("gpt2")

## the usages of this tiktoken is similar to tokenizer_v2 implimentated earlier via encode and decode method

In [ ]:
text = "Hello world, what is going on with you now? are u okay? <|endoftext|> Hari is from australia. He does good job but not well."

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers) 

[15496, 995, 11, 644, 318, 1016, 319, 351, 345, 783, 30, 389, 334, 8788, 30, 220, 50256, 2113, 72, 318, 422, 38132, 1373, 544, 13, 679, 857, 922, 1693, 475, 407, 880, 13]


In [ ]:
text_decoded = tokenizer.decode(integers)
print(text_decoded)

Hello world, what is going on with you now? are u okay? <|endoftext|> Hari is from australia. He does good job but not well.


## you may see one token integer as 50256. It is a special token called <|endoftext|> which is used by the model to indicate the end of the text.

# creaing ip/op target pairs
- for prediction word, we use something called contexxt size which says what is input side and its corresponsig op size
- eg: llm #are #good.
    - llm are #good.
    - llm are good.
- how do we get this?
- use context size say 4
- then input may be [1,2,3,4] and output [2,3,4,5] so,op for 1 will be 2 and for 1,2 be 3 and so on...
  


In [ ]:
# lets code
with open ('the-verdict.txt', 'r') as f:
    raw_text = f.read()

encoded_sample = tokenizer.encode(raw_text[50:])

# print(encoded_sample)
# print(tokenizer.decode(encoded_sample))

NameError: name 'tokenizer' is not defined

In [ ]:
context_size =  50

for i in range(0,context_size):
    context = encoded_sample[:i+1]
    desired_output = encoded_sample[i+1]
    print(tokenizer.decode(context), "-->", tokenizer.decode([desired_output]))
   

en --> ius
enius --> --
enius-- --> though
enius--though -->  a
enius--though a -->  good
enius--though a good -->  fellow
enius--though a good fellow -->  enough
enius--though a good fellow enough --> --
enius--though a good fellow enough-- --> so
enius--though a good fellow enough--so -->  it
enius--though a good fellow enough--so it -->  was
enius--though a good fellow enough--so it was -->  no
enius--though a good fellow enough--so it was no -->  great
enius--though a good fellow enough--so it was no great -->  surprise
enius--though a good fellow enough--so it was no great surprise -->  to
enius--though a good fellow enough--so it was no great surprise to -->  me
enius--though a good fellow enough--so it was no great surprise to me -->  to
enius--though a good fellow enough--so it was no great surprise to me to -->  hear
enius--though a good fellow enough--so it was no great surprise to me to hear -->  that
enius--though a good fellow enough--so it was no great surprise to me to h

# What are data loaders?
Dataset = organizes and exposes your data in a structured way. Think of it like a neatly arranged spreadsheet where each row is a sample (input + label) and each row knows how to be accessed and preprocessed.

DataLoader = takes that organized data and feeds it to your model efficiently, in batches, optionally shuffled, and can use multiple CPU threads to load and preprocess data in parallel so the GPU never sits idle waiting for data.

# Dataset, DataLoader, and Batch Size (PyTorch – LLM context)

## Big Picture
- **Dataset**: defines *what one training example looks like*
- **DataLoader**: defines *how training examples are grouped and fed to the model*
- **Batch size**: number of training examples processed **together** in one forward pass

---

## Dataset

A Dataset answers two questions:
1. How many samples are there?
2. What is the `i`-th sample?

In PyTorch, a Dataset typically implements:
- `__len__()` → total number of samples
- `__getitem__(idx)` → returns **one training example**

### In LLMs
One dataset item usually returns:
- `x`: input token IDs (length = `max_length`)
- `y`: target token IDs (same length, shifted by 1)

Example (conceptual):
```text
dataset[i] → (x_i, y_i)
x_i.shape = (max_length,)
y_i.shape = (max_length,)


Batch
┌───────────────────────────────┐
│ Seq 1: [t1  t2  ... t256]     │
│ Seq 2: [t5  t6  ... t260]     │
│ Seq 3: [t9  t10 ... t264]     │
│ Seq 4: [t13 t14 ... t268]     │
└───────────────────────────────┘



In [ ]:
!pip install torch

^C


In [ ]:
# %pip install torch --quiet

import torch
from torch.utils.data import Dataset, DataLoader

In [ ]:

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))#appends to the list the input chunk which is a tensor of length max_length
            self.target_ids.append(torch.tensor(target_chunk))#output chunk start from input start + 1 with length max_length

            # in another iteration the another input and output chunk will be added to the list

    def __len__(self):
        
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [ ]:
def create_dataLoader(txt,batch_size=4, stride=128, max_length=256,Shuffle=True,drop_last=True,num_workers=0):
    tokenizer  = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    # print(dataset[0])
    dataLoader = DataLoader(
        dataset,
        batch_size=batch_size,# this tells in a batch how many sequences will be there if it is 4 then it will have 4 sequences in a batch means 4 input sequences and 4 output sequences
        shuffle=Shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )
    return dataLoader
    


In [ ]:
with open ('the-verdict.txt', 'r') as f:
    text = f.read()
data_loader = create_dataLoader(text, batch_size=4, stride=128, max_length=256,Shuffle=True,drop_last=True,num_workers=0)

data_iter = iter(data_loader)#it will make a iteration of the data loader and returns a iterator
# to acess the data we need to use next

print(next(data_iter))
# print(next(data_iter))

# print(next(data_iter))
# print(next(data_iter))

# print(next(data_iter))
# print(next(data_iter))

[tensor([[  284,  1234,  8737,  ...,   290,   336,  8375],
        [ 3347, 27846,   503,  ...,  2951,   625,     0],
        [  262,  1633,   286,  ...,  1544,   373, 10574],
        [ 1459,   714,  1239,  ...,  1517,   484,  1444]]), tensor([[ 1234,  8737,   656,  ...,   336,  8375,   503],
        [27846,   503,  2048,  ...,   625,     0, 19672],
        [ 1633,   286, 24380,  ...,   373, 10574,    26],
        [  714,  1239,   423,  ...,   484,  1444,   616]])]
